In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [2]:
crash_path = "Motor_Vehicle_Collisions_Crashes.csv"
person_path = "Motor_Vehicle_Collisions_Person.csv"
vehicles_path = "Motor_Vehicle_Collisions_Vehicles.csv"

df_crashes = pd.read_csv(crash_path, dtype={'ZIP CODE': str})
df_person = pd.read_csv(person_path)
df_vehicles = pd.read_csv(vehicles_path, dtype={'VEHICLE_MODEL': str})

In [3]:
df_crashes.rename(columns={'CRASH DATE': 'CRASH_DATE', 'CRASH TIME': 'CRASH_TIME'}, inplace=True)

In [4]:
df_crashes['CRASH_DATE'] = pd.to_datetime(df_crashes['CRASH_DATE'])
df_person['CRASH_DATE'] = pd.to_datetime(df_person['CRASH_DATE'])
df_vehicles['CRASH_DATE'] = pd.to_datetime(df_vehicles['CRASH_DATE'])

df_crashes['CRASH_TIME'] = pd.to_datetime(df_crashes['CRASH_TIME'], format='%H:%M').dt.time
df_person['CRASH_TIME'] = pd.to_datetime(df_person['CRASH_TIME'], format='%H:%M').dt.time
df_vehicles['CRASH_TIME'] = pd.to_datetime(df_vehicles['CRASH_TIME'], format='%H:%M').dt.time

# PERSON CLEANING

In [5]:
df_person.drop(['UNIQUE_ID', 'PERSON_ID', 'VEHICLE_ID',
                 'EMOTIONAL_STATUS', 'BODILY_INJURY', 'COMPLAINT',
                 'CONTRIBUTING_FACTOR_1', 'CONTRIBUTING_FACTOR_2',
                 'PED_LOCATION', 'PED_ACTION'],
                 axis=1, inplace=True)

In [6]:
df_person

,COLLISION_ID,CRASH_DATE,CRASH_TIME,PERSON_TYPE,PERSON_INJURY,PERSON_AGE,EJECTION,POSITION_IN_VEHICLE,SAFETY_EQUIPMENT,PED_ROLE,PERSON_SEX
0,4229554,2019-10-26,09:43:00,Occupant,Unspecified,NaN,NaN,NaN,NaN,Registrant,U
1,4230587,2019-10-25,15:15:00,Occupant,Unspecified,33.0,Not Ejected,"Front passenger, if two or more persons, inclu...",Lap Belt & Harness,Passenger,F
2,4230550,2019-10-26,17:55:00,Occupant,Unspecified,55.0,NaN,NaN,NaN,Registrant,M
3,3565527,2016-11-21,13:05:00,Occupant,Unspecified,NaN,NaN,NaN,NaN,Notified Person,NaN
4,4231168,2019-10-25,11:16:00,Occupant,Unspecified,7.0,Not Ejected,Right rear passenger or motorcycle sidecar pas...,Lap Belt,Passenger,F
...,...,...,...,...,...,...,...,...,...,...,...
5296223,4705068,2024-02-25,09:05:00,Occupant,Unspecified,NaN,NaN,NaN,NaN,Registrant,U
5296224,4704919,2024-02-25,00:32:00,Occupant,Unspecified,24.0,Not Ejected,Driver,Unknown,Driver,M
5296225,4698398,2024-01-28,16:55:00,Occupant,Injured,50.0,Not Ejected,Driver,Unknown,Driver,F
5296226,4705556,2024-02-24,12:53:00,Occupant,Unspecified,63.0,NaN,NaN,NaN,Registrant,M


In [7]:
df_person.dropna(subset=['PERSON_SEX', 'PED_ROLE'], inplace=True)

df_person['SAFETY_EQUIPMENT'].fillna("Unknown", inplace=True)
df_person['SAFETY_EQUIPMENT'].replace('-', 'Unknown', inplace=True)

df_person['EJECTION'].fillna("Unknown", inplace=True)

df_person['POSITION_IN_VEHICLE'].fillna("Unknown", inplace=True)
df_person['POSITION_IN_VEHICLE'].replace('If one person is seated on another person&apos;s lap', 'If one person is seated on another persons lap', inplace=True)

df_person['POSITION_IN_VEHICLE'] = df_person['POSITION_IN_VEHICLE'].str.replace(',', '')

df_person.loc[(df_person['PERSON_AGE'] < 16) & (df_person['PED_ROLE'] == 'Driver'), 'PED_ROLE'] = 'Other'
df_person.loc[(df_person['PERSON_AGE'] < 16) & (df_person['PED_ROLE'] == 'Registrant'), 'PED_ROLE'] = 'Other'

df_person = df_person[(df_person['PERSON_AGE'] >= 1) & (df_person['PERSON_AGE'] <= 100)]

#df_person = df_person[df_person['PERSON_SEX'] != 'U']
#df_person = df_person[~((df_person['PERSON_AGE'] < 16) & (df_person['PED_ROLE'] == 'Driver'))] #Other
#df_person = df_person[~((df_person['PERSON_AGE'] < 16) & (df_person['PED_ROLE'] == 'Registrant'))]

In [8]:
count_df = df_person.groupby('COLLISION_ID').size().reset_index(name='NUM_PEOPLE')
count_df

,COLLISION_ID,NUM_PEOPLE
0,3405175,2
1,3405176,3
2,3405178,1
3,3405179,2
4,3405180,3
...,...,...
1252753,4705708,3
1252754,4705709,1
1252755,4705710,2
1252756,4705712,7


In [9]:
new_df = df_person.groupby(['COLLISION_ID', 'CRASH_DATE', 'CRASH_TIME']).agg(lambda x: list(x)).reset_index()
new_df

,COLLISION_ID,CRASH_DATE,CRASH_TIME,PERSON_TYPE,PERSON_INJURY,PERSON_AGE,EJECTION,POSITION_IN_VEHICLE,SAFETY_EQUIPMENT,PED_ROLE,PERSON_SEX
0,3405175,2016-03-12,14:10:00,"[Occupant, Occupant]","[Unspecified, Unspecified]","[32.0, 29.0]","[Not Ejected, Not Ejected]","[Driver, Driver]","[Lap Belt, Lap Belt]","[Driver, Driver]","[M, F]"
1,3405176,2016-03-13,18:30:00,"[Occupant, Occupant, Occupant]","[Unspecified, Unspecified, Unspecified]","[64.0, 47.0, 64.0]","[Unknown, Not Ejected, Not Ejected]","[Unknown, Driver, Driver]","[Unknown, Lap Belt & Harness, Lap Belt & Harness]","[Registrant, Driver, Driver]","[M, F, M]"
2,3405178,2016-03-14,10:20:00,[Occupant],[Unspecified],[30.0],[Not Ejected],[Driver],[Lap Belt & Harness],[Driver],[M]
3,3405179,2016-03-14,14:30:00,"[Occupant, Occupant]","[Unspecified, Unspecified]","[58.0, 32.0]","[Not Ejected, Not Ejected]","[Driver, Driver]","[Lap Belt & Harness, Lap Belt & Harness]","[Driver, Driver]","[M, M]"
4,3405180,2016-03-14,09:20:00,"[Occupant, Occupant, Occupant]","[Unspecified, Unspecified, Unspecified]","[52.0, 53.0, 59.0]","[Does Not Apply, Unknown, Does Not Apply]","[Driver, Unknown, Driver]","[Unknown, Unknown, Unknown]","[Driver, Registrant, Driver]","[M, M, M]"
...,...,...,...,...,...,...,...,...,...,...,...
1252753,4705708,2024-02-25,14:03:00,"[Occupant, Occupant, Occupant]","[Unspecified, Unspecified, Unspecified]","[52.0, 84.0, 52.0]","[Not Ejected, Unknown, Unknown]","[Driver, Unknown, Unknown]","[Unknown, Unknown, Unknown]","[Driver, Registrant, Registrant]","[F, F, F]"
1252754,4705709,2024-02-01,09:53:00,[Occupant],[Unspecified],[40.0],[Not Ejected],[Driver],[Lap Belt & Harness],[Driver],[M]
1252755,4705710,2024-02-02,16:43:00,"[Pedestrian, Occupant]","[Injured, Unspecified]","[15.0, 62.0]","[Unknown, Not Ejected]","[Unknown, Driver]","[Unknown, Unknown]","[Pedestrian, Driver]","[M, M]"
1252756,4705712,2024-01-05,23:52:00,"[Occupant, Occupant, Occupant, Occupant, Occup...","[Unspecified, Unspecified, Unspecified, Unspec...","[24.0, 39.0, 27.0, 32.0, 34.0, 30.0, 24.0]","[Not Ejected, Unknown, Not Ejected, Not Ejecte...",[Left rear passenger or rear passenger on a bi...,"[Unknown, Unknown, Lap Belt & Harness, Lap Bel...","[Passenger, Registrant, Passenger, Passenger, ...","[M, M, M, M, M, M, F]"


In [10]:
new_df.iloc[:, 3:] = new_df.iloc[:, 3:].apply(lambda x: [sorted(sublist) for sublist in x])
new_df

,COLLISION_ID,CRASH_DATE,CRASH_TIME,PERSON_TYPE,PERSON_INJURY,PERSON_AGE,EJECTION,POSITION_IN_VEHICLE,SAFETY_EQUIPMENT,PED_ROLE,PERSON_SEX
0,3405175,2016-03-12,14:10:00,"[Occupant, Occupant]","[Unspecified, Unspecified]","[29.0, 32.0]","[Not Ejected, Not Ejected]","[Driver, Driver]","[Lap Belt, Lap Belt]","[Driver, Driver]","[F, M]"
1,3405176,2016-03-13,18:30:00,"[Occupant, Occupant, Occupant]","[Unspecified, Unspecified, Unspecified]","[47.0, 64.0, 64.0]","[Not Ejected, Not Ejected, Unknown]","[Driver, Driver, Unknown]","[Lap Belt & Harness, Lap Belt & Harness, Unknown]","[Driver, Driver, Registrant]","[F, M, M]"
2,3405178,2016-03-14,10:20:00,[Occupant],[Unspecified],[30.0],[Not Ejected],[Driver],[Lap Belt & Harness],[Driver],[M]
3,3405179,2016-03-14,14:30:00,"[Occupant, Occupant]","[Unspecified, Unspecified]","[32.0, 58.0]","[Not Ejected, Not Ejected]","[Driver, Driver]","[Lap Belt & Harness, Lap Belt & Harness]","[Driver, Driver]","[M, M]"
4,3405180,2016-03-14,09:20:00,"[Occupant, Occupant, Occupant]","[Unspecified, Unspecified, Unspecified]","[52.0, 53.0, 59.0]","[Does Not Apply, Does Not Apply, Unknown]","[Driver, Driver, Unknown]","[Unknown, Unknown, Unknown]","[Driver, Driver, Registrant]","[M, M, M]"
...,...,...,...,...,...,...,...,...,...,...,...
1252753,4705708,2024-02-25,14:03:00,"[Occupant, Occupant, Occupant]","[Unspecified, Unspecified, Unspecified]","[52.0, 52.0, 84.0]","[Not Ejected, Unknown, Unknown]","[Driver, Unknown, Unknown]","[Unknown, Unknown, Unknown]","[Driver, Registrant, Registrant]","[F, F, F]"
1252754,4705709,2024-02-01,09:53:00,[Occupant],[Unspecified],[40.0],[Not Ejected],[Driver],[Lap Belt & Harness],[Driver],[M]
1252755,4705710,2024-02-02,16:43:00,"[Occupant, Pedestrian]","[Injured, Unspecified]","[15.0, 62.0]","[Not Ejected, Unknown]","[Driver, Unknown]","[Unknown, Unknown]","[Driver, Pedestrian]","[M, M]"
1252756,4705712,2024-01-05,23:52:00,"[Occupant, Occupant, Occupant, Occupant, Occup...","[Unspecified, Unspecified, Unspecified, Unspec...","[24.0, 24.0, 27.0, 30.0, 32.0, 34.0, 39.0]","[Not Ejected, Not Ejected, Not Ejected, Not Ej...","[Driver, Driver, Front passenger if two or mor...","[Lap Belt & Harness, Lap Belt & Harness, Lap B...","[Driver, Driver, Passenger, Passenger, Passeng...","[F, M, M, M, M, M, M]"


In [11]:
new_df['AVERAGE_AGE'] = new_df['PERSON_AGE'].apply(lambda x : np.mean(x))

def change(arr):
    return [0 if sex == 'M' else 1 for sex in arr]

new_df['PERSON_SEX'] = new_df['PERSON_SEX'].apply(change)

new_df['AVERAGE_SEX'] = new_df['PERSON_SEX'].apply(lambda x : np.mean(x))

new_df['PED_PRESENT'] = new_df['PERSON_TYPE'].apply(lambda x: 1 if 'Pedestrian' in x else 0)

new_df['CYC_PRESENT'] = new_df['PERSON_TYPE'].apply(lambda x: 1 if 'Bicyclist' in x else 0)

new_df['EJECTED'] = new_df['EJECTION'].apply(lambda x: 1 if any(item in ['Ejected', 'Partially Ejected', 'Trapped'] for item in x) else 0)

new_df

,COLLISION_ID,CRASH_DATE,CRASH_TIME,PERSON_TYPE,PERSON_INJURY,PERSON_AGE,EJECTION,POSITION_IN_VEHICLE,SAFETY_EQUIPMENT,PED_ROLE,PERSON_SEX,AVERAGE_AGE,AVERAGE_SEX,PED_PRESENT,CYC_PRESENT,EJECTED
0,3405175,2016-03-12,14:10:00,"[Occupant, Occupant]","[Unspecified, Unspecified]","[29.0, 32.0]","[Not Ejected, Not Ejected]","[Driver, Driver]","[Lap Belt, Lap Belt]","[Driver, Driver]","[1, 0]",30.500000,0.500000,0,0,0
1,3405176,2016-03-13,18:30:00,"[Occupant, Occupant, Occupant]","[Unspecified, Unspecified, Unspecified]","[47.0, 64.0, 64.0]","[Not Ejected, Not Ejected, Unknown]","[Driver, Driver, Unknown]","[Lap Belt & Harness, Lap Belt & Harness, Unknown]","[Driver, Driver, Registrant]","[1, 0, 0]",58.333333,0.333333,0,0,0
2,3405178,2016-03-14,10:20:00,[Occupant],[Unspecified],[30.0],[Not Ejected],[Driver],[Lap Belt & Harness],[Driver],[0],30.000000,0.000000,0,0,0
3,3405179,2016-03-14,14:30:00,"[Occupant, Occupant]","[Unspecified, Unspecified]","[32.0, 58.0]","[Not Ejected, Not Ejected]","[Driver, Driver]","[Lap Belt & Harness, Lap Belt & Harness]","[Driver, Driver]","[0, 0]",45.000000,0.000000,0,0,0
4,3405180,2016-03-14,09:20:00,"[Occupant, Occupant, Occupant]","[Unspecified, Unspecified, Unspecified]","[52.0, 53.0, 59.0]","[Does Not Apply, Does Not Apply, Unknown]","[Driver, Driver, Unknown]","[Unknown, Unknown, Unknown]","[Driver, Driver, Registrant]","[0, 0, 0]",54.666667,0.000000,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1252753,4705708,2024-02-25,14:03:00,"[Occupant, Occupant, Occupant]","[Unspecified, Unspecified, Unspecified]","[52.0, 52.0, 84.0]","[Not Ejected, Unknown, Unknown]","[Driver, Unknown, Unknown]","[Unknown, Unknown, Unknown]","[Driver, Registrant, Registrant]","[1, 1, 1]",62.666667,1.000000,0,0,0
1252754,4705709,2024-02-01,09:53:00,[Occupant],[Unspecified],[40.0],[Not Ejected],[Driver],[Lap Belt & Harness],[Driver],[0],40.000000,0.000000,0,0,0
1252755,4705710,2024-02-02,16:43:00,"[Occupant, Pedestrian]","[Injured, Unspecified]","[15.0, 62.0]","[Not Ejected, Unknown]","[Driver, Unknown]","[Unknown, Unknown]","[Driver, Pedestrian]","[0, 0]",38.500000,0.000000,1,0,0
1252756,4705712,2024-01-05,23:52:00,"[Occupant, Occupant, Occupant, Occupant, Occup...","[Unspecified, Unspecified, Unspecified, Unspec...","[24.0, 24.0, 27.0, 30.0, 32.0, 34.0, 39.0]","[Not Ejected, Not Ejected, Not Ejected, Not Ej...","[Driver, Driver, Front passenger if two or mor...","[Lap Belt & Harness, Lap Belt & Harness, Lap B...","[Driver, Driver, Passenger, Passenger, Passeng...","[1, 0, 0, 0, 0, 0, 0]",30.000000,0.142857,0,0,0


In [12]:
new_person_df = pd.merge(new_df, count_df, on='COLLISION_ID')
new_person_df

,COLLISION_ID,CRASH_DATE,CRASH_TIME,PERSON_TYPE,PERSON_INJURY,PERSON_AGE,EJECTION,POSITION_IN_VEHICLE,SAFETY_EQUIPMENT,PED_ROLE,PERSON_SEX,AVERAGE_AGE,AVERAGE_SEX,PED_PRESENT,CYC_PRESENT,EJECTED,NUM_PEOPLE
0,3405175,2016-03-12,14:10:00,"[Occupant, Occupant]","[Unspecified, Unspecified]","[29.0, 32.0]","[Not Ejected, Not Ejected]","[Driver, Driver]","[Lap Belt, Lap Belt]","[Driver, Driver]","[1, 0]",30.500000,0.500000,0,0,0,2
1,3405176,2016-03-13,18:30:00,"[Occupant, Occupant, Occupant]","[Unspecified, Unspecified, Unspecified]","[47.0, 64.0, 64.0]","[Not Ejected, Not Ejected, Unknown]","[Driver, Driver, Unknown]","[Lap Belt & Harness, Lap Belt & Harness, Unknown]","[Driver, Driver, Registrant]","[1, 0, 0]",58.333333,0.333333,0,0,0,3
2,3405178,2016-03-14,10:20:00,[Occupant],[Unspecified],[30.0],[Not Ejected],[Driver],[Lap Belt & Harness],[Driver],[0],30.000000,0.000000,0,0,0,1
3,3405179,2016-03-14,14:30:00,"[Occupant, Occupant]","[Unspecified, Unspecified]","[32.0, 58.0]","[Not Ejected, Not Ejected]","[Driver, Driver]","[Lap Belt & Harness, Lap Belt & Harness]","[Driver, Driver]","[0, 0]",45.000000,0.000000,0,0,0,2
4,3405180,2016-03-14,09:20:00,"[Occupant, Occupant, Occupant]","[Unspecified, Unspecified, Unspecified]","[52.0, 53.0, 59.0]","[Does Not Apply, Does Not Apply, Unknown]","[Driver, Driver, Unknown]","[Unknown, Unknown, Unknown]","[Driver, Driver, Registrant]","[0, 0, 0]",54.666667,0.000000,0,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1252753,4705708,2024-02-25,14:03:00,"[Occupant, Occupant, Occupant]","[Unspecified, Unspecified, Unspecified]","[52.0, 52.0, 84.0]","[Not Ejected, Unknown, Unknown]","[Driver, Unknown, Unknown]","[Unknown, Unknown, Unknown]","[Driver, Registrant, Registrant]","[1, 1, 1]",62.666667,1.000000,0,0,0,3
1252754,4705709,2024-02-01,09:53:00,[Occupant],[Unspecified],[40.0],[Not Ejected],[Driver],[Lap Belt & Harness],[Driver],[0],40.000000,0.000000,0,0,0,1
1252755,4705710,2024-02-02,16:43:00,"[Occupant, Pedestrian]","[Injured, Unspecified]","[15.0, 62.0]","[Not Ejected, Unknown]","[Driver, Unknown]","[Unknown, Unknown]","[Driver, Pedestrian]","[0, 0]",38.500000,0.000000,1,0,0,2
1252756,4705712,2024-01-05,23:52:00,"[Occupant, Occupant, Occupant, Occupant, Occup...","[Unspecified, Unspecified, Unspecified, Unspec...","[24.0, 24.0, 27.0, 30.0, 32.0, 34.0, 39.0]","[Not Ejected, Not Ejected, Not Ejected, Not Ej...","[Driver, Driver, Front passenger if two or mor...","[Lap Belt & Harness, Lap Belt & Harness, Lap B...","[Driver, Driver, Passenger, Passenger, Passeng...","[1, 0, 0, 0, 0, 0, 0]",30.000000,0.142857,0,0,0,7


# CRASH CLEANING

In [13]:
df_crashes.drop(['ON STREET NAME', 'CROSS STREET NAME', 'OFF STREET NAME',
                 'NUMBER OF PEDESTRIANS INJURED', 'NUMBER OF PEDESTRIANS KILLED',
                 'NUMBER OF CYCLIST INJURED', 'NUMBER OF CYCLIST KILLED',
                 'NUMBER OF MOTORIST INJURED', 'NUMBER OF MOTORIST KILLED',
                 'CONTRIBUTING FACTOR VEHICLE 3', 'CONTRIBUTING FACTOR VEHICLE 4', 'CONTRIBUTING FACTOR VEHICLE 5',
                 'VEHICLE TYPE CODE 3', 'VEHICLE TYPE CODE 4', 'VEHICLE TYPE CODE 5'],
                 axis=1, inplace=True)

In [14]:
df_crashes = df_crashes[df_crashes['ZIP CODE'] != '10000']
df_crashes = df_crashes[df_crashes['ZIP CODE'] != '     ']
df_crashes.dropna(subset=['ZIP CODE'], inplace=True)

df_crashes.dropna(subset=['BOROUGH'], inplace=True)

df_crashes = df_crashes.dropna(subset=['LOCATION'])
df_crashes = df_crashes[df_crashes['LOCATION'] != '(0.0, 0.0)']

In [15]:
df_crashes['ZIP CODE'] = df_crashes['ZIP CODE'].astype(int)

In [16]:
df_crashes['CONTRIBUTING FACTOR VEHICLE 1'] = df_crashes['CONTRIBUTING FACTOR VEHICLE 1'].str.lower()
df_crashes['CONTRIBUTING FACTOR VEHICLE 2'] = df_crashes['CONTRIBUTING FACTOR VEHICLE 2'].str.lower()

df_crashes['CONTRIBUTING FACTOR VEHICLE 1'] = df_crashes['CONTRIBUTING FACTOR VEHICLE 1'].replace('illnes', 'illness')
df_crashes['CONTRIBUTING FACTOR VEHICLE 2'] = df_crashes['CONTRIBUTING FACTOR VEHICLE 2'].replace('illnes', 'illness')

df_crashes['CONTRIBUTING FACTOR VEHICLE 1'].fillna("unspecified", inplace=True)
df_crashes['CONTRIBUTING FACTOR VEHICLE 2'].fillna("unspecified", inplace=True)

values_to_drop = ['80', '1']
df_crashes.drop(df_crashes[df_crashes['CONTRIBUTING FACTOR VEHICLE 1'].isin(values_to_drop)].index, inplace=True)
df_crashes.drop(df_crashes[df_crashes['CONTRIBUTING FACTOR VEHICLE 2'].isin(values_to_drop)].index, inplace=True)

df_crashes['VEHICLE TYPE CODE 1'] = df_crashes['VEHICLE TYPE CODE 1'].str.lower()
df_crashes['VEHICLE TYPE CODE 2'] = df_crashes['VEHICLE TYPE CODE 2'].str.lower()

df_crashes['VEHICLE TYPE CODE 1'] = df_crashes['VEHICLE TYPE CODE 1'].replace('sport utility / station wagon', 'station wagon/sport utility vehicle')
df_crashes['VEHICLE TYPE CODE 2'] = df_crashes['VEHICLE TYPE CODE 2'].replace('sport utility / station wagon', 'station wagon/sport utility vehicle')

df_crashes['VEHICLE TYPE CODE 1'].fillna("unknown", inplace=True)
df_crashes['VEHICLE TYPE CODE 2'].fillna("unknown", inplace=True)

counts_cf1 = df_crashes['CONTRIBUTING FACTOR VEHICLE 1'].value_counts(dropna=False)
rows_to_remove_cf1 = counts_cf1[counts_cf1 < 100].index
df_crashes = df_crashes[~df_crashes['CONTRIBUTING FACTOR VEHICLE 1'].isin(rows_to_remove_cf1)]

counts_cf2 = df_crashes['CONTRIBUTING FACTOR VEHICLE 2'].value_counts(dropna=False)
rows_to_remove_cf2 = counts_cf2[counts_cf2 < 100].index
df_crashes = df_crashes[~df_crashes['CONTRIBUTING FACTOR VEHICLE 2'].isin(rows_to_remove_cf2)]

counts_vt1 = df_crashes['VEHICLE TYPE CODE 1'].value_counts(dropna=False)
rows_to_remove_vt1 = counts_vt1[counts_vt1 < 100].index
df_crashes = df_crashes[~df_crashes['VEHICLE TYPE CODE 1'].isin(rows_to_remove_vt1)]

counts_vt2 = df_crashes['VEHICLE TYPE CODE 2'].value_counts(dropna=False)
rows_to_remove_vt2 = counts_vt2[counts_vt2 < 100].index
df_crashes = df_crashes[~df_crashes['VEHICLE TYPE CODE 2'].isin(rows_to_remove_vt2)]

In [17]:
two_wheel_vehicles = ['e-bike', 'e-scooter', 'moped', 'bike', 'motorbike', 'scooter', 'motorcycle']
large_vehicles = ['flat bed', 'refrigerated van', 'dump', 'tanker', 'tow truck/wrecker']

def check_large(row): 
    return any([
        row['VEHICLE TYPE CODE 1'] in large_vehicles, 
        row['VEHICLE TYPE CODE 2'] in large_vehicles
    ])

df_crashes['LARGE_VEHICLE'] = df_crashes.apply(check_large, axis=1).astype(int)


def check_two_wheel(row):
    return any([
        row['VEHICLE TYPE CODE 1'] in two_wheel_vehicles, 
        row['VEHICLE TYPE CODE 2'] in two_wheel_vehicles
    ])

df_crashes['TWO_WHEEL_VEHICLE'] = df_crashes.apply(check_two_wheel, axis=1).astype(int)

In [18]:
df_crash_people = pd.merge(df_crashes, new_person_df, on=['COLLISION_ID', 'CRASH_DATE', 'CRASH_TIME'], how='inner')
df_crash_people

,CRASH_DATE,CRASH_TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,CONTRIBUTING FACTOR VEHICLE 1,...,POSITION_IN_VEHICLE,SAFETY_EQUIPMENT,PED_ROLE,PERSON_SEX,AVERAGE_AGE,AVERAGE_SEX,PED_PRESENT,CYC_PRESENT,EJECTED,NUM_PEOPLE
0,2021-09-11,09:35:00,BROOKLYN,11208,40.667202,-73.866500,"(40.667202, -73.8665)",0.0,0.0,unspecified,...,"[Unknown, Unknown]","[Unknown, Unknown]","[Driver, Registrant]","[1, 1]",28.000000,1.000000,0,0,0,2
1,2021-12-14,08:13:00,BROOKLYN,11233,40.683304,-73.917274,"(40.683304, -73.917274)",0.0,0.0,unspecified,...,[Unknown],[Unknown],[Pedestrian],[0],46.000000,0.000000,1,0,0,1
2,2021-12-14,08:17:00,BRONX,10475,40.868160,-73.831480,"(40.86816, -73.83148)",2.0,0.0,unspecified,...,"[Driver, Driver, Unknown]","[Lap Belt & Harness, Lap Belt & Harness, Unknown]","[Driver, Driver, Registrant]","[1, 1, 0]",34.000000,0.666667,0,0,0,3
3,2021-12-14,14:58:00,MANHATTAN,10017,40.751440,-73.973970,"(40.75144, -73.97397)",0.0,0.0,passing too closely,...,"[Driver, Driver]","[Lap Belt, Lap Belt]","[Driver, Driver]","[0, 0]",29.500000,0.000000,0,0,0,2
4,2021-12-14,16:50:00,QUEENS,11413,40.675884,-73.755770,"(40.675884, -73.75577)",0.0,0.0,turning improperly,...,"[Driver, Driver, Unknown, Unknown]","[Lap Belt & Harness, Lap Belt & Harness, Unkno...","[Driver, Driver, Registrant, Registrant]","[1, 1, 0, 0]",46.000000,0.500000,0,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755667,2024-02-19,10:00:00,QUEENS,11377,40.742020,-73.892660,"(40.74202, -73.89266)",0.0,0.0,driver inattention/distraction,...,[Unknown],[Unknown],[Registrant],[0],57.000000,0.000000,0,0,0,1
755668,2024-02-25,08:25:00,BROOKLYN,11211,40.706276,-73.955060,"(40.706276, -73.95506)",0.0,0.0,driver inattention/distraction,...,"[Unknown, Unknown, Unknown, Unknown]","[Unknown, Unknown, Unknown, Unknown]","[Driver, Driver, Registrant, Registrant]","[0, 0, 0, 0]",36.000000,0.000000,0,0,0,4
755669,2024-02-25,14:50:00,QUEENS,11370,40.773808,-73.892900,"(40.773808, -73.8929)",2.0,0.0,failure to yield right-of-way,...,"[Driver, Driver, Front passenger if two or mor...","[Unknown, Unknown, Unknown, Unknown, Unknown]","[Driver, Other, Passenger, Passenger, Registrant]","[1, 1, 1, 0, 0]",38.400000,0.600000,0,0,0,5
755670,2024-02-21,12:00:00,BROOKLYN,11207,40.657660,-73.897050,"(40.65766, -73.89705)",0.0,0.0,unspecified,...,"[Driver, Unknown, Unknown]","[Unknown, Unknown, Unknown]","[Driver, Registrant, Registrant]","[1, 1, 1]",37.666667,1.000000,0,0,0,3


# Vehicle Cleaning

In [19]:
df_vehicles.drop(['UNIQUE_ID', 'VEHICLE_ID', 'VEHICLE_MODEL',
                  'TRAVEL_DIRECTION',
                  'PUBLIC_PROPERTY_DAMAGE_TYPE',
                  'VEHICLE_DAMAGE_1', 'VEHICLE_DAMAGE_2', 'VEHICLE_DAMAGE_3',
                  'CONTRIBUTING_FACTOR_1', 'CONTRIBUTING_FACTOR_2'],
                  axis=1, inplace=True)

In [20]:
# STATE_REGISTRATION: remove NaN, remove non states
# VEHICLE_MAKE: remove row will less than 100
# VEHICLE_YEAR: remove NaN
# TRAVEL_DIRECTION: remove as values are equal useless
# VEHICLE_OCCUPANTS: remove less than 100
# DRIVER_SEX: NaN to U
# DRIVER_LICENSE_STATUS: remove NaN
# PRE_CRASH: NaN to Other, Other* to Other
# POINT_OF_IMPACT: NaN to Unknown
# PUBLIC_PROPERTY_DAMAGE: NaN to Unspecified

In [21]:
counts_vm = df_vehicles['VEHICLE_MAKE'].value_counts(dropna=False)
rows_to_remove_vm = counts_vm[counts_vm < 100].index
df_vehicles = df_vehicles[~df_vehicles['VEHICLE_MAKE'].isin(rows_to_remove_vm)]

df_vehicles.dropna(subset=['VEHICLE_YEAR'], inplace=True)

counts_vo = df_vehicles['VEHICLE_OCCUPANTS'].value_counts(dropna=False)
rows_to_remove_vo = counts_vo[counts_vo < 100].index
df_vehicles = df_vehicles[~df_vehicles['VEHICLE_OCCUPANTS'].isin(rows_to_remove_vo)]

df_vehicles['DRIVER_SEX'].fillna("U", inplace=True)

df_vehicles.dropna(subset=['DRIVER_LICENSE_STATUS'], inplace=True)

df_vehicles['PRE_CRASH'] = df_vehicles['PRE_CRASH'].replace('Other*', 'Other')
df_vehicles['PRE_CRASH'].fillna("Other", inplace=True)

df_vehicles['POINT_OF_IMPACT'].fillna("Unknown", inplace=True)

df_vehicles['PUBLIC_PROPERTY_DAMAGE'].fillna("Unknown", inplace=True)

df_vehicles['VEHICLE_TYPE'] = df_vehicles['VEHICLE_TYPE'].str.lower()

df_vehicles['VEHICLE_TYPE'] = df_vehicles['VEHICLE_TYPE'].replace('sport utility / station wagon', 'station wagon/sport utility vehicle')

df_vehicles['VEHICLE_TYPE'].fillna("unknown", inplace=True)

counts_vt = df_vehicles['VEHICLE_TYPE'].value_counts(dropna=False)
rows_to_remove_vt = counts_vt[counts_vt < 100].index
df_vehicles = df_vehicles[~df_vehicles['VEHICLE_TYPE'].isin(rows_to_remove_vt)]

df_vehicles['VEHICLE_MAKE'] = df_vehicles['VEHICLE_MAKE'].str.upper()
df_vehicles['VEHICLE_MAKE'] = df_vehicles['VEHICLE_MAKE'].str.replace(r'\s+', '', regex=True)
df_vehicles['VEHICLE_MAKE'].fillna("UNKNOWN", inplace=True)

df_vehicles['VEHICLE_DAMAGE'].fillna("Other", inplace=True)

df_vehicles['VEHICLE_OCCUPANTS'] = df_vehicles['VEHICLE_OCCUPANTS'].replace(0, 1)
df_vehicles.dropna(subset=['VEHICLE_OCCUPANTS'], inplace=True)

In [22]:
state_abbreviations = [
    'AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 
    'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 
    'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 
    'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 
    'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY'
]

# Filtering the DataFrame to keep rows with state abbreviations
df_vehicles = df_vehicles[df_vehicles['STATE_REGISTRATION'].isin(state_abbreviations)]
df_vehicles = df_vehicles[df_vehicles['DRIVER_LICENSE_JURISDICTION'].isin(state_abbreviations)]


In [23]:
count_df_vehicles = df_vehicles.groupby('COLLISION_ID').size().reset_index(name='NUM_VEHICLES')
count_df_vehicles

,COLLISION_ID,NUM_VEHICLES
0,3405176,2
1,3405179,2
2,3405180,2
3,3405183,2
4,3405184,2
...,...,...
1077179,4705708,1
1077180,4705709,1
1077181,4705710,1
1077182,4705712,1


In [24]:
new_df_vehicles = df_vehicles.groupby(['COLLISION_ID', 'CRASH_DATE', 'CRASH_TIME']).agg(lambda x: list(x)).reset_index()
new_df_vehicles

,COLLISION_ID,CRASH_DATE,CRASH_TIME,STATE_REGISTRATION,VEHICLE_TYPE,VEHICLE_MAKE,VEHICLE_YEAR,VEHICLE_OCCUPANTS,DRIVER_SEX,DRIVER_LICENSE_STATUS,DRIVER_LICENSE_JURISDICTION,PRE_CRASH,POINT_OF_IMPACT,VEHICLE_DAMAGE,PUBLIC_PROPERTY_DAMAGE
0,3405176,2016-03-13,18:30:00,"[NY, NJ]","[taxi, 4 dr sedan]","[TOYT-CAR/SUV, MERZ-CAR/SUV]","[2015.0, 2007.0]","[1.0, 1.0]","[M, F]","[Licensed, Licensed]","[NY, NJ]","[Going Straight Ahead, Making Left Turn]","[Left Front Bumper, Right Front Bumper]","[Left Front Bumper, Right Front Bumper]","[N, N]"
1,3405179,2016-03-14,14:30:00,"[NY, NY]","[station wagon/sport utility vehicle, station ...","[HOND-CAR/SUV, FORD-CAR/SUV]","[2015.0, 2011.0]","[1.0, 1.0]","[M, M]","[Licensed, Licensed]","[NY, NY]","[Stopped in Traffic, Going Straight Ahead]","[Left Side Doors, Right Side Doors]","[Left Side Doors, Right Side Doors]","[N, N]"
2,3405180,2016-03-14,09:20:00,"[NY, NY]","[4 dr sedan, 4 dr sedan]","[TOYT-CAR/SUV, TOYT-CAR/SUV]","[2015.0, 2014.0]","[1.0, 1.0]","[M, M]","[Licensed, Licensed]","[NY, NY]","[Entering Parked Position, Going Straight Ahead]","[Right Front Quarter Panel, Left Side Doors]","[Right Front Quarter Panel, Left Side Doors]","[N, N]"
3,3405183,2016-03-14,08:00:00,"[NJ, NJ]","[station wagon/sport utility vehicle, unknown]","[ACUR-CAR/SUV, MACK]","[2011.0, 2006.0]","[1.0, 1.0]","[M, M]","[Licensed, Licensed]","[NJ, NJ]","[Changing Lanes, Going Straight Ahead]","[Right Front Quarter Panel, Left Rear Quarter ...","[Right Front Quarter Panel, Left Front Quarter...","[N, N]"
4,3405184,2016-03-14,11:10:00,"[NY, NY]","[4 dr sedan, pick-up truck]","[TOYT-CAR/SUV, CHEV-CAR/SUV]","[2014.0, 2014.0]","[1.0, 1.0]","[M, M]","[Licensed, Licensed]","[NY, NY]","[Making Right Turn, Passing]","[Left Front Bumper, Right Front Bumper]","[Left Front Bumper, Right Front Bumper]","[N, N]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1077179,4705708,2024-02-25,14:03:00,[NY],[station wagon/sport utility vehicle],[CHEV-CAR/SUV],[2015.0],[1.0],[F],[Licensed],[NY],[Parked],[Left Front Bumper],[Left Front Bumper],[N]
1077180,4705709,2024-02-01,09:53:00,[NY],[sedan],[FORD-CAR/SUV],[2012.0],[1.0],[M],[Licensed],[NY],[Going Straight Ahead],[Right Rear Quarter Panel],[Right Rear Quarter Panel],[Unspecified]
1077181,4705710,2024-02-02,16:43:00,[NY],[van],[FORD],[2007.0],[1.0],[M],[Licensed],[NY],[Passing],[Right Front Quarter Panel],[No Damage],[N]
1077182,4705712,2024-01-05,23:52:00,[AZ],[sedan],[FORD-CAR/SUV],[2022.0],[3.0],[M],[Unlicensed],[NY],[Going Straight Ahead],[Center Front End],[Center Front End],[N]


In [25]:
new_df_vehicles.iloc[:, 3:] = new_df_vehicles.iloc[:, 3:].apply(lambda x: [sorted(sublist) for sublist in x])
new_df_vehicles

,COLLISION_ID,CRASH_DATE,CRASH_TIME,STATE_REGISTRATION,VEHICLE_TYPE,VEHICLE_MAKE,VEHICLE_YEAR,VEHICLE_OCCUPANTS,DRIVER_SEX,DRIVER_LICENSE_STATUS,DRIVER_LICENSE_JURISDICTION,PRE_CRASH,POINT_OF_IMPACT,VEHICLE_DAMAGE,PUBLIC_PROPERTY_DAMAGE
0,3405176,2016-03-13,18:30:00,"[NJ, NY]","[4 dr sedan, taxi]","[MERZ-CAR/SUV, TOYT-CAR/SUV]","[2007.0, 2015.0]","[1.0, 1.0]","[F, M]","[Licensed, Licensed]","[NJ, NY]","[Going Straight Ahead, Making Left Turn]","[Left Front Bumper, Right Front Bumper]","[Left Front Bumper, Right Front Bumper]","[N, N]"
1,3405179,2016-03-14,14:30:00,"[NY, NY]","[station wagon/sport utility vehicle, station ...","[FORD-CAR/SUV, HOND-CAR/SUV]","[2011.0, 2015.0]","[1.0, 1.0]","[M, M]","[Licensed, Licensed]","[NY, NY]","[Going Straight Ahead, Stopped in Traffic]","[Left Side Doors, Right Side Doors]","[Left Side Doors, Right Side Doors]","[N, N]"
2,3405180,2016-03-14,09:20:00,"[NY, NY]","[4 dr sedan, 4 dr sedan]","[TOYT-CAR/SUV, TOYT-CAR/SUV]","[2014.0, 2015.0]","[1.0, 1.0]","[M, M]","[Licensed, Licensed]","[NY, NY]","[Entering Parked Position, Going Straight Ahead]","[Left Side Doors, Right Front Quarter Panel]","[Left Side Doors, Right Front Quarter Panel]","[N, N]"
3,3405183,2016-03-14,08:00:00,"[NJ, NJ]","[station wagon/sport utility vehicle, unknown]","[ACUR-CAR/SUV, MACK]","[2006.0, 2011.0]","[1.0, 1.0]","[M, M]","[Licensed, Licensed]","[NJ, NJ]","[Changing Lanes, Going Straight Ahead]","[Left Rear Quarter Panel, Right Front Quarter ...","[Left Front Quarter Panel, Right Front Quarter...","[N, N]"
4,3405184,2016-03-14,11:10:00,"[NY, NY]","[4 dr sedan, pick-up truck]","[CHEV-CAR/SUV, TOYT-CAR/SUV]","[2014.0, 2014.0]","[1.0, 1.0]","[M, M]","[Licensed, Licensed]","[NY, NY]","[Making Right Turn, Passing]","[Left Front Bumper, Right Front Bumper]","[Left Front Bumper, Right Front Bumper]","[N, N]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1077179,4705708,2024-02-25,14:03:00,[NY],[station wagon/sport utility vehicle],[CHEV-CAR/SUV],[2015.0],[1.0],[F],[Licensed],[NY],[Parked],[Left Front Bumper],[Left Front Bumper],[N]
1077180,4705709,2024-02-01,09:53:00,[NY],[sedan],[FORD-CAR/SUV],[2012.0],[1.0],[M],[Licensed],[NY],[Going Straight Ahead],[Right Rear Quarter Panel],[Right Rear Quarter Panel],[Unspecified]
1077181,4705710,2024-02-02,16:43:00,[NY],[van],[FORD],[2007.0],[1.0],[M],[Licensed],[NY],[Passing],[Right Front Quarter Panel],[No Damage],[N]
1077182,4705712,2024-01-05,23:52:00,[AZ],[sedan],[FORD-CAR/SUV],[2022.0],[3.0],[M],[Unlicensed],[NY],[Going Straight Ahead],[Center Front End],[Center Front End],[N]


In [26]:
new_df_vehicles['VEHICLE_OCCUPANTS'] = new_df_vehicles['VEHICLE_OCCUPANTS'].apply(lambda x : np.mean(x))

def change(arr):
    return [0 if sex == 'M' else 1 for sex in arr]

def prop_damage(arr):
    return [0 if i == 'Y' else 1 for i in arr]

new_df_vehicles['DRIVER_SEX'] = new_df_vehicles['DRIVER_SEX'].apply(change)

new_df_vehicles['AVERAGE_DRIVER_SEX'] = new_df_vehicles['DRIVER_SEX'].apply(lambda x : np.mean(x))

new_df_vehicles['STATUS_L'] = new_df_vehicles['DRIVER_LICENSE_STATUS'].apply(lambda x: 1 if 'Licensed' in x else 0)

new_df_vehicles['STATUS_U'] = new_df_vehicles['DRIVER_LICENSE_STATUS'].apply(lambda x: 1 if 'Unlicensed' in x else 0)

new_df_vehicles['STATUS_P'] = new_df_vehicles['DRIVER_LICENSE_STATUS'].apply(lambda x: 1 if 'Permit' in x else 0)

new_df_vehicles['PUBLIC_PROPERTY_DAMAGE'] = new_df_vehicles['PUBLIC_PROPERTY_DAMAGE'].apply(prop_damage)

new_df_vehicles

,COLLISION_ID,CRASH_DATE,CRASH_TIME,STATE_REGISTRATION,VEHICLE_TYPE,VEHICLE_MAKE,VEHICLE_YEAR,VEHICLE_OCCUPANTS,DRIVER_SEX,DRIVER_LICENSE_STATUS,DRIVER_LICENSE_JURISDICTION,PRE_CRASH,POINT_OF_IMPACT,VEHICLE_DAMAGE,PUBLIC_PROPERTY_DAMAGE,AVERAGE_DRIVER_SEX,STATUS_L,STATUS_U,STATUS_P
0,3405176,2016-03-13,18:30:00,"[NJ, NY]","[4 dr sedan, taxi]","[MERZ-CAR/SUV, TOYT-CAR/SUV]","[2007.0, 2015.0]",1.0,"[1, 0]","[Licensed, Licensed]","[NJ, NY]","[Going Straight Ahead, Making Left Turn]","[Left Front Bumper, Right Front Bumper]","[Left Front Bumper, Right Front Bumper]","[1, 1]",0.5,1,0,0
1,3405179,2016-03-14,14:30:00,"[NY, NY]","[station wagon/sport utility vehicle, station ...","[FORD-CAR/SUV, HOND-CAR/SUV]","[2011.0, 2015.0]",1.0,"[0, 0]","[Licensed, Licensed]","[NY, NY]","[Going Straight Ahead, Stopped in Traffic]","[Left Side Doors, Right Side Doors]","[Left Side Doors, Right Side Doors]","[1, 1]",0.0,1,0,0
2,3405180,2016-03-14,09:20:00,"[NY, NY]","[4 dr sedan, 4 dr sedan]","[TOYT-CAR/SUV, TOYT-CAR/SUV]","[2014.0, 2015.0]",1.0,"[0, 0]","[Licensed, Licensed]","[NY, NY]","[Entering Parked Position, Going Straight Ahead]","[Left Side Doors, Right Front Quarter Panel]","[Left Side Doors, Right Front Quarter Panel]","[1, 1]",0.0,1,0,0
3,3405183,2016-03-14,08:00:00,"[NJ, NJ]","[station wagon/sport utility vehicle, unknown]","[ACUR-CAR/SUV, MACK]","[2006.0, 2011.0]",1.0,"[0, 0]","[Licensed, Licensed]","[NJ, NJ]","[Changing Lanes, Going Straight Ahead]","[Left Rear Quarter Panel, Right Front Quarter ...","[Left Front Quarter Panel, Right Front Quarter...","[1, 1]",0.0,1,0,0
4,3405184,2016-03-14,11:10:00,"[NY, NY]","[4 dr sedan, pick-up truck]","[CHEV-CAR/SUV, TOYT-CAR/SUV]","[2014.0, 2014.0]",1.0,"[0, 0]","[Licensed, Licensed]","[NY, NY]","[Making Right Turn, Passing]","[Left Front Bumper, Right Front Bumper]","[Left Front Bumper, Right Front Bumper]","[1, 1]",0.0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1077179,4705708,2024-02-25,14:03:00,[NY],[station wagon/sport utility vehicle],[CHEV-CAR/SUV],[2015.0],1.0,[1],[Licensed],[NY],[Parked],[Left Front Bumper],[Left Front Bumper],[1],1.0,1,0,0
1077180,4705709,2024-02-01,09:53:00,[NY],[sedan],[FORD-CAR/SUV],[2012.0],1.0,[0],[Licensed],[NY],[Going Straight Ahead],[Right Rear Quarter Panel],[Right Rear Quarter Panel],[1],0.0,1,0,0
1077181,4705710,2024-02-02,16:43:00,[NY],[van],[FORD],[2007.0],1.0,[0],[Licensed],[NY],[Passing],[Right Front Quarter Panel],[No Damage],[1],0.0,1,0,0
1077182,4705712,2024-01-05,23:52:00,[AZ],[sedan],[FORD-CAR/SUV],[2022.0],3.0,[0],[Unlicensed],[NY],[Going Straight Ahead],[Center Front End],[Center Front End],[1],0.0,0,1,0


In [27]:
new_vehicles_df = pd.merge(new_df_vehicles, count_df_vehicles, on='COLLISION_ID')
new_vehicles_df

,COLLISION_ID,CRASH_DATE,CRASH_TIME,STATE_REGISTRATION,VEHICLE_TYPE,VEHICLE_MAKE,VEHICLE_YEAR,VEHICLE_OCCUPANTS,DRIVER_SEX,DRIVER_LICENSE_STATUS,DRIVER_LICENSE_JURISDICTION,PRE_CRASH,POINT_OF_IMPACT,VEHICLE_DAMAGE,PUBLIC_PROPERTY_DAMAGE,AVERAGE_DRIVER_SEX,STATUS_L,STATUS_U,STATUS_P,NUM_VEHICLES
0,3405176,2016-03-13,18:30:00,"[NJ, NY]","[4 dr sedan, taxi]","[MERZ-CAR/SUV, TOYT-CAR/SUV]","[2007.0, 2015.0]",1.0,"[1, 0]","[Licensed, Licensed]","[NJ, NY]","[Going Straight Ahead, Making Left Turn]","[Left Front Bumper, Right Front Bumper]","[Left Front Bumper, Right Front Bumper]","[1, 1]",0.5,1,0,0,2
1,3405179,2016-03-14,14:30:00,"[NY, NY]","[station wagon/sport utility vehicle, station ...","[FORD-CAR/SUV, HOND-CAR/SUV]","[2011.0, 2015.0]",1.0,"[0, 0]","[Licensed, Licensed]","[NY, NY]","[Going Straight Ahead, Stopped in Traffic]","[Left Side Doors, Right Side Doors]","[Left Side Doors, Right Side Doors]","[1, 1]",0.0,1,0,0,2
2,3405180,2016-03-14,09:20:00,"[NY, NY]","[4 dr sedan, 4 dr sedan]","[TOYT-CAR/SUV, TOYT-CAR/SUV]","[2014.0, 2015.0]",1.0,"[0, 0]","[Licensed, Licensed]","[NY, NY]","[Entering Parked Position, Going Straight Ahead]","[Left Side Doors, Right Front Quarter Panel]","[Left Side Doors, Right Front Quarter Panel]","[1, 1]",0.0,1,0,0,2
3,3405183,2016-03-14,08:00:00,"[NJ, NJ]","[station wagon/sport utility vehicle, unknown]","[ACUR-CAR/SUV, MACK]","[2006.0, 2011.0]",1.0,"[0, 0]","[Licensed, Licensed]","[NJ, NJ]","[Changing Lanes, Going Straight Ahead]","[Left Rear Quarter Panel, Right Front Quarter ...","[Left Front Quarter Panel, Right Front Quarter...","[1, 1]",0.0,1,0,0,2
4,3405184,2016-03-14,11:10:00,"[NY, NY]","[4 dr sedan, pick-up truck]","[CHEV-CAR/SUV, TOYT-CAR/SUV]","[2014.0, 2014.0]",1.0,"[0, 0]","[Licensed, Licensed]","[NY, NY]","[Making Right Turn, Passing]","[Left Front Bumper, Right Front Bumper]","[Left Front Bumper, Right Front Bumper]","[1, 1]",0.0,1,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1077179,4705708,2024-02-25,14:03:00,[NY],[station wagon/sport utility vehicle],[CHEV-CAR/SUV],[2015.0],1.0,[1],[Licensed],[NY],[Parked],[Left Front Bumper],[Left Front Bumper],[1],1.0,1,0,0,1
1077180,4705709,2024-02-01,09:53:00,[NY],[sedan],[FORD-CAR/SUV],[2012.0],1.0,[0],[Licensed],[NY],[Going Straight Ahead],[Right Rear Quarter Panel],[Right Rear Quarter Panel],[1],0.0,1,0,0,1
1077181,4705710,2024-02-02,16:43:00,[NY],[van],[FORD],[2007.0],1.0,[0],[Licensed],[NY],[Passing],[Right Front Quarter Panel],[No Damage],[1],0.0,1,0,0,1
1077182,4705712,2024-01-05,23:52:00,[AZ],[sedan],[FORD-CAR/SUV],[2022.0],3.0,[0],[Unlicensed],[NY],[Going Straight Ahead],[Center Front End],[Center Front End],[1],0.0,0,1,0,1


In [28]:
df_crash_master = pd.merge(df_crash_people, new_vehicles_df, on=['COLLISION_ID', 'CRASH_DATE', 'CRASH_TIME'], how='inner')
df_crash_master

,CRASH_DATE,CRASH_TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,CONTRIBUTING FACTOR VEHICLE 1,...,DRIVER_LICENSE_JURISDICTION,PRE_CRASH,POINT_OF_IMPACT,VEHICLE_DAMAGE,PUBLIC_PROPERTY_DAMAGE,AVERAGE_DRIVER_SEX,STATUS_L,STATUS_U,STATUS_P,NUM_VEHICLES
0,2021-09-11,09:35:00,BROOKLYN,11208,40.667202,-73.866500,"(40.667202, -73.8665)",0.0,0.0,unspecified,...,[NC],[Parked],[Left Rear Quarter Panel],[Left Rear Quarter Panel],[1],1.0,1,0,0,1
1,2021-12-14,08:17:00,BRONX,10475,40.868160,-73.831480,"(40.86816, -73.83148)",2.0,0.0,unspecified,...,"[NY, NY]","[Going Straight Ahead, Going Straight Ahead]","[Center Front End, Right Front Quarter Panel]","[Center Front End, Right Front Quarter Panel]","[1, 1]",0.5,1,0,0,2
2,2021-12-14,14:58:00,MANHATTAN,10017,40.751440,-73.973970,"(40.75144, -73.97397)",0.0,0.0,passing too closely,...,"[NY, NY]","[Changing Lanes, Going Straight Ahead]","[Left Rear Bumper, Right Front Bumper]","[Left Rear Bumper, Right Front Bumper]","[1, 1]",0.0,1,1,0,2
3,2021-12-14,16:50:00,QUEENS,11413,40.675884,-73.755770,"(40.675884, -73.75577)",0.0,0.0,turning improperly,...,"[NY, NY]","[Going Straight Ahead, Making Right Turn]","[Left Front Quarter Panel, Right Side Doors]","[Left Front Quarter Panel, Right Side Doors]","[1, 1]",0.5,1,0,0,2
4,2021-12-14,23:10:00,QUEENS,11434,40.666840,-73.789410,"(40.66684, -73.78941)",2.0,0.0,reaction to uninvolved vehicle,...,"[NY, NY]","[Going Straight Ahead, Going Straight Ahead]","[Center Front End, Center Front End]","[Demolished, Other]","[1, 1]",0.5,1,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
628347,2024-02-23,12:38:00,BRONX,10459,40.820972,-73.892845,"(40.820972, -73.892845)",0.0,0.0,unspecified,...,[NY],[Going Straight Ahead],[Right Front Bumper],[Left Front Bumper],[1],0.0,1,0,0,1
628348,2024-02-25,08:25:00,BROOKLYN,11211,40.706276,-73.955060,"(40.706276, -73.95506)",0.0,0.0,driver inattention/distraction,...,"[NY, NY]","[Parked, Parked]","[Center Back End, Left Rear Bumper]","[Center Back End, Left Rear Bumper]","[1, 1]",0.0,1,0,0,2
628349,2024-02-25,14:50:00,QUEENS,11370,40.773808,-73.892900,"(40.773808, -73.8929)",2.0,0.0,failure to yield right-of-way,...,"[NY, NY]","[Going Straight Ahead, Making Left Turn]","[Center Front End, Right Front Quarter Panel]","[Center Front End, Right Front Bumper]","[1, 1]",0.5,1,0,0,2
628350,2024-02-21,12:00:00,BROOKLYN,11207,40.657660,-73.897050,"(40.65766, -73.89705)",0.0,0.0,unspecified,...,[GA],[Parked],[Left Front Bumper],[Left Front Quarter Panel],[1],1.0,1,0,0,1


In [29]:
def label_output(row):
    if row['NUMBER OF PERSONS KILLED'] >= 1:
        return 'Killed'
    elif row['NUMBER OF PERSONS INJURED'] >= 1:
        return 'Injured'
    else: 
        return 'Unspecified'

In [30]:
df_crash_master['OUTPUT'] = df_crash_master.apply(label_output, axis=1)
df_crash_master.head()

,CRASH_DATE,CRASH_TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,CONTRIBUTING FACTOR VEHICLE 1,...,PRE_CRASH,POINT_OF_IMPACT,VEHICLE_DAMAGE,PUBLIC_PROPERTY_DAMAGE,AVERAGE_DRIVER_SEX,STATUS_L,STATUS_U,STATUS_P,NUM_VEHICLES,OUTPUT
0,2021-09-11,09:35:00,BROOKLYN,11208,40.667202,-73.86650,"(40.667202, -73.8665)",0.0,0.0,unspecified,...,[Parked],[Left Rear Quarter Panel],[Left Rear Quarter Panel],[1],1.0,1,0,0,1,Unspecified
1,2021-12-14,08:17:00,BRONX,10475,40.868160,-73.83148,"(40.86816, -73.83148)",2.0,0.0,unspecified,...,"[Going Straight Ahead, Going Straight Ahead]","[Center Front End, Right Front Quarter Panel]","[Center Front End, Right Front Quarter Panel]","[1, 1]",0.5,1,0,0,2,Injured
2,2021-12-14,14:58:00,MANHATTAN,10017,40.751440,-73.97397,"(40.75144, -73.97397)",0.0,0.0,passing too closely,...,"[Changing Lanes, Going Straight Ahead]","[Left Rear Bumper, Right Front Bumper]","[Left Rear Bumper, Right Front Bumper]","[1, 1]",0.0,1,1,0,2,Unspecified
3,2021-12-14,16:50:00,QUEENS,11413,40.675884,-73.75577,"(40.675884, -73.75577)",0.0,0.0,turning improperly,...,"[Going Straight Ahead, Making Right Turn]","[Left Front Quarter Panel, Right Side Doors]","[Left Front Quarter Panel, Right Side Doors]","[1, 1]",0.5,1,0,0,2,Unspecified
4,2021-12-14,23:10:00,QUEENS,11434,40.666840,-73.78941,"(40.66684, -73.78941)",2.0,0.0,reaction to uninvolved vehicle,...,"[Going Straight Ahead, Going Straight Ahead]","[Center Front End, Center Front End]","[Demolished, Other]","[1, 1]",0.5,1,0,0,2,Injured


In [31]:
df_crash_master.drop(columns=['NUMBER OF PERSONS INJURED', 'NUMBER OF PERSONS KILLED', 'PERSON_INJURY'], inplace=True)

In [32]:
df_crash_master.head()

,CRASH_DATE,CRASH_TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,COLLISION_ID,...,PRE_CRASH,POINT_OF_IMPACT,VEHICLE_DAMAGE,PUBLIC_PROPERTY_DAMAGE,AVERAGE_DRIVER_SEX,STATUS_L,STATUS_U,STATUS_P,NUM_VEHICLES,OUTPUT
0,2021-09-11,09:35:00,BROOKLYN,11208,40.667202,-73.86650,"(40.667202, -73.8665)",unspecified,unspecified,4456314,...,[Parked],[Left Rear Quarter Panel],[Left Rear Quarter Panel],[1],1.0,1,0,0,1,Unspecified
1,2021-12-14,08:17:00,BRONX,10475,40.868160,-73.83148,"(40.86816, -73.83148)",unspecified,unspecified,4486660,...,"[Going Straight Ahead, Going Straight Ahead]","[Center Front End, Right Front Quarter Panel]","[Center Front End, Right Front Quarter Panel]","[1, 1]",0.5,1,0,0,2,Injured
2,2021-12-14,14:58:00,MANHATTAN,10017,40.751440,-73.97397,"(40.75144, -73.97397)",passing too closely,unspecified,4486519,...,"[Changing Lanes, Going Straight Ahead]","[Left Rear Bumper, Right Front Bumper]","[Left Rear Bumper, Right Front Bumper]","[1, 1]",0.0,1,1,0,2,Unspecified
3,2021-12-14,16:50:00,QUEENS,11413,40.675884,-73.75577,"(40.675884, -73.75577)",turning improperly,unspecified,4487127,...,"[Going Straight Ahead, Making Right Turn]","[Left Front Quarter Panel, Right Side Doors]","[Left Front Quarter Panel, Right Side Doors]","[1, 1]",0.5,1,0,0,2,Unspecified
4,2021-12-14,23:10:00,QUEENS,11434,40.666840,-73.78941,"(40.66684, -73.78941)",reaction to uninvolved vehicle,unspecified,4486635,...,"[Going Straight Ahead, Going Straight Ahead]","[Center Front End, Center Front End]","[Demolished, Other]","[1, 1]",0.5,1,0,0,2,Injured


In [33]:
df_crash_master.to_pickle('master_crash.pkl')

In [34]:
master_crash_path = "master_crash.pkl"

df_master = pd.read_pickle(master_crash_path)

In [35]:
df_master

,CRASH_DATE,CRASH_TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,COLLISION_ID,...,PRE_CRASH,POINT_OF_IMPACT,VEHICLE_DAMAGE,PUBLIC_PROPERTY_DAMAGE,AVERAGE_DRIVER_SEX,STATUS_L,STATUS_U,STATUS_P,NUM_VEHICLES,OUTPUT
0,2021-09-11,09:35:00,BROOKLYN,11208,40.667202,-73.866500,"(40.667202, -73.8665)",unspecified,unspecified,4456314,...,[Parked],[Left Rear Quarter Panel],[Left Rear Quarter Panel],[1],1.0,1,0,0,1,Unspecified
1,2021-12-14,08:17:00,BRONX,10475,40.868160,-73.831480,"(40.86816, -73.83148)",unspecified,unspecified,4486660,...,"[Going Straight Ahead, Going Straight Ahead]","[Center Front End, Right Front Quarter Panel]","[Center Front End, Right Front Quarter Panel]","[1, 1]",0.5,1,0,0,2,Injured
2,2021-12-14,14:58:00,MANHATTAN,10017,40.751440,-73.973970,"(40.75144, -73.97397)",passing too closely,unspecified,4486519,...,"[Changing Lanes, Going Straight Ahead]","[Left Rear Bumper, Right Front Bumper]","[Left Rear Bumper, Right Front Bumper]","[1, 1]",0.0,1,1,0,2,Unspecified
3,2021-12-14,16:50:00,QUEENS,11413,40.675884,-73.755770,"(40.675884, -73.75577)",turning improperly,unspecified,4487127,...,"[Going Straight Ahead, Making Right Turn]","[Left Front Quarter Panel, Right Side Doors]","[Left Front Quarter Panel, Right Side Doors]","[1, 1]",0.5,1,0,0,2,Unspecified
4,2021-12-14,23:10:00,QUEENS,11434,40.666840,-73.789410,"(40.66684, -73.78941)",reaction to uninvolved vehicle,unspecified,4486635,...,"[Going Straight Ahead, Going Straight Ahead]","[Center Front End, Center Front End]","[Demolished, Other]","[1, 1]",0.5,1,0,0,2,Injured
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
628347,2024-02-23,12:38:00,BRONX,10459,40.820972,-73.892845,"(40.820972, -73.892845)",unspecified,unspecified,4705614,...,[Going Straight Ahead],[Right Front Bumper],[Left Front Bumper],[1],0.0,1,0,0,1,Unspecified
628348,2024-02-25,08:25:00,BROOKLYN,11211,40.706276,-73.955060,"(40.706276, -73.95506)",driver inattention/distraction,unspecified,4705193,...,"[Parked, Parked]","[Center Back End, Left Rear Bumper]","[Center Back End, Left Rear Bumper]","[1, 1]",0.0,1,0,0,2,Unspecified
628349,2024-02-25,14:50:00,QUEENS,11370,40.773808,-73.892900,"(40.773808, -73.8929)",failure to yield right-of-way,unspecified,4705104,...,"[Going Straight Ahead, Making Left Turn]","[Center Front End, Right Front Quarter Panel]","[Center Front End, Right Front Bumper]","[1, 1]",0.5,1,0,0,2,Injured
628350,2024-02-21,12:00:00,BROOKLYN,11207,40.657660,-73.897050,"(40.65766, -73.89705)",unspecified,unspecified,4705537,...,[Parked],[Left Front Bumper],[Left Front Quarter Panel],[1],1.0,1,0,0,1,Unspecified


In [40]:
df_crash_master.to_csv('master_crash.csv', index=False)

In [41]:
master_crash_path_csv = "master_crash.csv"

df_master_csv = pd.read_csv(master_crash_path_csv)

In [42]:
df_master_csv

,CRASH_DATE,CRASH_TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,COLLISION_ID,...,PRE_CRASH,POINT_OF_IMPACT,VEHICLE_DAMAGE,PUBLIC_PROPERTY_DAMAGE,AVERAGE_DRIVER_SEX,STATUS_L,STATUS_U,STATUS_P,NUM_VEHICLES,OUTPUT
0,2021-09-11,09:35:00,BROOKLYN,11208,40.667202,-73.866500,"(40.667202, -73.8665)",unspecified,unspecified,4456314,...,['Parked'],['Left Rear Quarter Panel'],['Left Rear Quarter Panel'],[1],1.0,1,0,0,1,Unspecified
1,2021-12-14,08:17:00,BRONX,10475,40.868160,-73.831480,"(40.86816, -73.83148)",unspecified,unspecified,4486660,...,"['Going Straight Ahead', 'Going Straight Ahead']","['Center Front End', 'Right Front Quarter Panel']","['Center Front End', 'Right Front Quarter Panel']","[1, 1]",0.5,1,0,0,2,Injured
2,2021-12-14,14:58:00,MANHATTAN,10017,40.751440,-73.973970,"(40.75144, -73.97397)",passing too closely,unspecified,4486519,...,"['Changing Lanes', 'Going Straight Ahead']","['Left Rear Bumper', 'Right Front Bumper']","['Left Rear Bumper', 'Right Front Bumper']","[1, 1]",0.0,1,1,0,2,Unspecified
3,2021-12-14,16:50:00,QUEENS,11413,40.675884,-73.755770,"(40.675884, -73.75577)",turning improperly,unspecified,4487127,...,"['Going Straight Ahead', 'Making Right Turn']","['Left Front Quarter Panel', 'Right Side Doors']","['Left Front Quarter Panel', 'Right Side Doors']","[1, 1]",0.5,1,0,0,2,Unspecified
4,2021-12-14,23:10:00,QUEENS,11434,40.666840,-73.789410,"(40.66684, -73.78941)",reaction to uninvolved vehicle,unspecified,4486635,...,"['Going Straight Ahead', 'Going Straight Ahead']","['Center Front End', 'Center Front End']","['Demolished', 'Other']","[1, 1]",0.5,1,0,0,2,Injured
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
628347,2024-02-23,12:38:00,BRONX,10459,40.820972,-73.892845,"(40.820972, -73.892845)",unspecified,unspecified,4705614,...,['Going Straight Ahead'],['Right Front Bumper'],['Left Front Bumper'],[1],0.0,1,0,0,1,Unspecified
628348,2024-02-25,08:25:00,BROOKLYN,11211,40.706276,-73.955060,"(40.706276, -73.95506)",driver inattention/distraction,unspecified,4705193,...,"['Parked', 'Parked']","['Center Back End', 'Left Rear Bumper']","['Center Back End', 'Left Rear Bumper']","[1, 1]",0.0,1,0,0,2,Unspecified
628349,2024-02-25,14:50:00,QUEENS,11370,40.773808,-73.892900,"(40.773808, -73.8929)",failure to yield right-of-way,unspecified,4705104,...,"['Going Straight Ahead', 'Making Left Turn']","['Center Front End', 'Right Front Quarter Panel']","['Center Front End', 'Right Front Bumper']","[1, 1]",0.5,1,0,0,2,Injured
628350,2024-02-21,12:00:00,BROOKLYN,11207,40.657660,-73.897050,"(40.65766, -73.89705)",unspecified,unspecified,4705537,...,['Parked'],['Left Front Bumper'],['Left Front Quarter Panel'],[1],1.0,1,0,0,1,Unspecified
